In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [2]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result


In [3]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']


In [4]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [5]:
x_train,x_test,y_train,y_test = load_data(test_size=0.25)

In [6]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [7]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [8]:
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [11]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from matplotlib.pyplot import specgram
import pandas as pd
from sklearn.metrics import confusion_matrix
import IPython.display as ipd
from IPython.display import Audio

In [ ]:
audio_path = 'data/Actor_'

In [45]:
dir_list = os.listdir('data')
dir_list.sort()
actor = []
emotion = []
gender = []
path = []
for i in dir_list:
    fname = os.listdir('data/' + i)
    for f in fname:
        part = f.split('.')[0].split('-')
        actor.append(int(part[1]))
        emotion.append(int(part[2]))
        temp = int(part[6])
        if temp%2 == 0:
            temp = "female"
        else:
            temp = "male"
        gender.append(temp)
        path.append('data/' + i + '/' + f)
        

        
RAV_df = pd.DataFrame(emotion)
RAV_df = RAV_df.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
RAV_df = pd.concat([pd.DataFrame(gender),RAV_df],axis=1)
RAV_df = pd.concat([pd.DataFrame(actor),RAV_df],axis=1)
RAV_df.columns = ['actor','gender','emotion']  
RAV_df = pd.concat([RAV_df,pd.DataFrame(path, columns = ['path'])],axis=1)
RAV_df.emotion.value_counts()

neutral     288
disgust     192
sad         192
happy       192
fear        192
angry       192
surprise    192
Name: emotion, dtype: int64

In [46]:
RAV_df.head()

,actor,gender,emotion,path
0,1,male,neutral,data/Actor_01/03-01-01-01-01-01-01.wav
1,1,male,neutral,data/Actor_01/03-01-01-01-01-02-01.wav
2,1,male,neutral,data/Actor_01/03-01-01-01-02-01-01.wav
3,1,male,neutral,data/Actor_01/03-01-01-01-02-02-01.wav
4,1,male,neutral,data/Actor_01/03-01-02-01-01-01-01.wav
